In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky,GRU_NCL,LambdaLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [3]:
## change ##
batch_size = 16
clip = .5

Prepare data

In [4]:
train_gen = SequenceGenSpec(train,Is2D=False)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False,Is2D=False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [49]:
convNets = Sequential(Dropout(0.5),
                      ConvBatchLeaky(1,4,(5,5),stride=(2,2)),
                      Dropout(0.5),
                      ConvBatchLeaky(4,8,(5,5),stride=(2,2)),
                      Dropout(0.5),
                      ConvBatchLeaky(8,16,(3,3),stride=(1,2)),
                      Dropout(0.5),
                      ConvBatchLeaky(16,32,(3,3),stride=(1,2)),
                      Dropout(0.5),
                      LambdaLayer(lambda x:x.transpose(2,3).contiguous().view(-1,32*47,26)),
                      GRU_NCL(32*47,32*47,batch_first=True,bidirectional=True,returnH=True)
                     )

In [50]:
linear = Sequential(Dropout(0.5),Linear(32*47,1))

In [51]:
model = CNN_RNN2seq_transpose(convNets,linear).to('cuda:0')

In [52]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [53]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [54]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:3.005897369522315, val_loss:3.338738441467285
epoch:1, train_loss:2.8510098663660197, val_loss:2.9846718311309814
epoch:2, train_loss:2.6302003631225, val_loss:2.8274521827697754
epoch:3, train_loss:2.424418965784403, val_loss:2.5752949714660645
epoch:4, train_loss:2.255917690694332, val_loss:2.4499130249023438
Training completed in 151.46714282035828s


In [55]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip,lossBest=2.4499)

epoch:0, train_loss:2.1790047391102862, val_loss:2.432323932647705
epoch:1, train_loss:2.1132785425736356, val_loss:2.438343048095703
epoch:2, train_loss:2.1458513690875125, val_loss:2.430865526199341
epoch:3, train_loss:2.137428566813469, val_loss:2.480020761489868
epoch:4, train_loss:2.1171455795948324, val_loss:2.4351489543914795
Training completed in 152.56863355636597s


Submission

In [56]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [57]:
test_gen = SequenceGenSpecTest(submission.seg_id.tolist(),Is2D=False)
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [58]:
submission.iloc[:,1] = predict(model,test_gen)

In [59]:
submission.to_csv('../Submission/sub_spec_3d_transpose.csv',index=False)